In [4]:
import geopandas as gpd
import pandas as pd


# analyse van data geocoded sample
- Van 2109 rijen in de geocoded sample ontbreekt de begintijd_iso. Alle kolommen zijn verschoven. Deze rijen zijn voor nu buiten beschouwing gelaten. 
- Sommige schepen hebben een v18_Laadvermogen van 0.0. 
- Er zitten geen afmetingen van de schepen in de dataset. Wel kunnen deze worden afgeleid uit de scheepsklasses

### De data ziet er zo uit:

In [8]:

data = gpd.read_file("data\ivs\ivs-2024-geocoded-sample.gpkg")
data.head()

,Jaarmaand,Jaar,Maand,Weeknr,v05_06_begindt_evenement_iso,v05_06_Begindt_evenement,UNLO_herkomst,UNLO_bestemming,v15_1_Scheepstype_RWS,SK_CODE,v18_Laadvermogen,v28_Beladingscode,v38_Vervoerd_gewicht,v30_4_Containers_TEU_S,nstr_nw,nst2007_nw,geometry
0,1812,2018,12,50,2018-12-10T14:00:00+01:00,10 december 2018 14:00:00 uur,NLZOU,NLBON,1,M2,690.0,1.0,NaN,NaN,None,None,"LINESTRING (4.50000 52.11667, 5.81667 51.03333)"
1,2305,2023,5,18,2023-05-05T21:00:00+02:00,05 mei 2023 21:00:00 uur,NLRTM,DEDUI,2,M6,1530.0,7.0,1300000.0,0.0,3,7.2,"LINESTRING (4.50000 51.91667, 6.75000 51.46667)"
2,1901,2019,1,2,2019-01-10T14:00:00+01:00,10 januari 2019 14:00:00 uur,NLUTC,BEANR,1,M8,3150.0,7.0,3070000.0,NaN,6,3.5,"LINESTRING (5.13333 52.08333, 4.41667 51.21667)"
3,2003,2020,3,11,2020-03-10T07:00:00+01:00,10 maart 2020 07:00:00 uur,BEGNK,DELUE,1,M4,930.0,7.0,931000.0,0.0,6,3.5,"LINESTRING (5.50000 50.96667, 7.52452 51.61812)"
4,2304,2023,4,16,2023-04-20T15:00:00+02:00,20 april 2023 15:00:00 uur,NLHAR,NLWTE,1,M12,450.0,1.0,NaN,0.0,None,None,"LINESTRING (5.43333 53.18333, 5.22147 53.36644)"


### Data in februari 2019 is versprongen:

In [41]:
verschoven_data = data[data['v05_06_begindt_evenement_iso'].apply(lambda x: x[10])!='T']
verschoven_data.head()


,Jaarmaand,Jaar,Maand,Weeknr,v05_06_begindt_evenement_iso,v05_06_Begindt_evenement,UNLO_herkomst,UNLO_bestemming,v15_1_Scheepstype_RWS,SK_CODE,v18_Laadvermogen,v28_Beladingscode,v38_Vervoerd_gewicht,v30_4_Containers_TEU_S,nstr_nw,nst2007_nw,geometry
137,2102,2021,2,5,02 februari 2021 09:00:00 uur,BEOLN,NLAMS,1,M8,2800,1.0,NaN,0.0,NaN,None,None,None
173,2101,2021,1,4,25 januari 2021 17:00:00 uur,BEWIK,NLBRS,1,M8,2410,1.0,NaN,0.0,NaN,None,None,None
346,2101,2021,1,2,17 januari 2021 05:00:00 uur,BEANR,NLRTM,2,M10,4240,7.0,806000.0,0.0,3.0,7.2,None,None
397,2101,2021,1,4,27 januari 2021 11:00:00 uur,DERES,BEANR,1,M5,1060,1.0,NaN,0.0,NaN,None,None,None
514,2102,2021,2,7,19 februari 2021 21:00:00 uur,DEOFF,NLAMS,1,M9,3780,1.0,NaN,0.0,NaN,None,None,None


### Hier wat algemene informatie over de data

In [42]:
#info over de data
print(f"aantal_metingen van nl naar nl: {len(data[data['UNLO_herkomst'].str.startswith('NL') & data['UNLO_bestemming'].str.startswith('NL')])}")
print(f"tijd loopt van {data['Jaar'].min()} tot {data['Jaar'].max()}")
print(f"aantal metingen in 2023: {len(data[data['Jaar']==2023])}")
print(f"aantal metingen in 2023 december: {len(data[(data['Jaar']==2023)&(data['Maand']==12)])}")
print(f"aantal metingen 1 december 2023: {len(data[data['v05_06_begindt_evenement_iso'].str.startswith('2023-12-01')])}")

aantal_metingen van nl naar nl: 47024
tijd loopt van 2018 tot 2024
aantal metingen in 2023: 15190
aantal metingen in 2023 december: 1068
aantal metingen 1 december 2023: 38


# Inlezen Parquet data:
- deze data bevat geen geoinformatie
- De parquet data wat een tussentijds resultaat. Hier is geometrie aan toegevoegd in het bestand ivs-2024-geocoded.gpkg.


In [5]:
data_parquet = pd.read_parquet('data\ivs\ivs-2024-one-file.parquet')
data_parquet.loc[0]

,Jaarmaand,Jaar,Maand,Weeknr,v05_06_begindt_evenement_iso,v05_06_Begindt_evenement,UNLO_herkomst,UNLO_bestemming,v15_1_Scheepstype_RWS,SK_CODE,v18_Laadvermogen,v28_Beladingscode,v38_Vervoerd_gewicht,v30_4_Containers_TEU_S,nstr_nw,nst2007_nw
2965,2103,2021,3,9,2021-03-03T11:00:00+01:00,03 maart 2021 11:00:00 uur,DEANE,NLAMS,1,M8,3260.0,7.0,1957000.0,0.0,4,14.2
4326,2103,2021,3,9,2021-03-04T11:00:00+01:00,04 maart 2021 11:00:00 uur,NLRTM,NLTLB,3,M2,570.0,7.0,50000.0,19.0,<NA>,<NA>
4422,2103,2021,3,9,2021-03-04T12:00:00+01:00,04 maart 2021 12:00:00 uur,FRAUS,BEANR,2,M6,1640.0,7.0,924000.0,0.0,7,8.3
5217,2103,2021,3,9,2021-03-05T03:00:00+01:00,05 maart 2021 03:00:00 uur,NLVEG,NLRTM,1,M7,2000.0,7.0,1013136.0,96.0,9,12.1
5287,2103,2021,3,9,2021-03-05T06:00:00+01:00,05 maart 2021 06:00:00 uur,NLRTM,NLMOE,2,M8,3000.0,7.0,500000.0,0.0,9,20.0
